In [13]:
# Setup
import os
import numpy as np # For general mathematical operations
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision
import matplotlib.pyplot as plt # For plotting the results
from torchsummary import summary # For model summaries
from torch.utils.tensorboard import SummaryWriter # For writing into tensorboard
import nrrd # For reading and manipulating nrrd files
from glob import glob
import torch.nn.functional as F
%load_ext autoreload
%autoreload 3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Develop focal loss

Specifically solve the alpha issue

In [65]:
torch.concat([alpha for _ in range(a)], dim=-1).shape

torch.Size([3600000])

In [64]:
a = 1
[a:=shape*a for shape in orig_input_shape]
a

360000

In [161]:
alpha = torch.Tensor(
[0.5, 1.0, 4.0, 1.0, 4.0, 4.0, 1.0, 1.0, 3.0, 3.0]
)  # TODO: focal loss weights per channels from the paper

weights = torch.ones(dims)
alpha_transformed = (weights.transpose(1,-1)*alpha).transpose(1,-1).view(-1)


gamma = 2.0
dims = [2, 10, 20, 30, 30]

targets = torch.rand(dims)
inputs = (torch.rand(dims) + targets) / 2

# Add error specifically to channel 0
inputs[0,...] = torch.rand(dims[1:])

orig_input_shape = inputs.shape
# flatten label and prediction tensors
inputs = inputs.view(-1)
targets = targets.view(-1)
# first compute binary cross-entropy

BCE = F.binary_cross_entropy(inputs, targets, weight=weights_transformed, reduction="mean")

BCE_EXP = torch.exp(-BCE)
focal_loss = (1 - BCE_EXP) ** gamma * BCE


focal_loss


tensor(1.2058)

In [ ]:
test = torch.rand((2,3,2,2)).round(decimals=2)

In [145]:
w = torch.Tensor([1,2,3])
weights = torch.ones_like(test)

weights = (weights.transpose(1, -1)*w).transpose(1,-1)
print(test)
print(test.view(-1))
print(weights.view(-1))

tensor([[[[0.6300, 0.6700],
          [0.2500, 0.8200]],

         [[0.2700, 0.6600],
          [0.8800, 0.7900]],

         [[0.2000, 0.2200],
          [0.3000, 0.3600]]],


        [[[0.3300, 0.8700],
          [0.4700, 0.0400]],

         [[0.9600, 0.0000],
          [0.8200, 0.7600]],

         [[0.5200, 0.4400],
          [0.5000, 0.4500]]]])
tensor([0.6300, 0.6700, 0.2500, 0.8200, 0.2700, 0.6600, 0.8800, 0.7900, 0.2000,
        0.2200, 0.3000, 0.3600, 0.3300, 0.8700, 0.4700, 0.0400, 0.9600, 0.0000,
        0.8200, 0.7600, 0.5200, 0.4400, 0.5000, 0.4500])
tensor([1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3., 1., 1., 1., 1., 2., 2.,
        2., 2., 3., 3., 3., 3.])
